In [1]:
from  tiny.package import *
from  tiny.util import *
print('begin')
install = get_drop_list_for_install()

install[install.package=='000533a8532cde925ccb4d232c0c6453']



2018-09-14 15:08:47,768 util_log.py[33] INFO Begin to run get_drop_list_for_install with:[], {}


begin


2018-09-14 15:09:27,120 util_log.py[38] INFO cost:39.35 sec: ==='get_drop_list_for_install' end ([], {}) 


,package,0
0,000533a8532cde925ccb4d232c0c6453,0


In [4]:
ls = ['000533a8532cde925ccb4d232c0c6453']
columns = [ col for col in install.columns if col in install.package.values ]
print(columns)

[]


In [11]:
if '000533a8532cde925ccb4d232c0c6453' in install.package.values:
    print('yes')


In [7]:
columns = [ col for col in install.columns ]
print(columns[:4])

['package', 0]


In [12]:
vectorizer = CountVectorizer()
cntTf_app = vectorizer.fit_transform(apps)

str(cntTf_app[:3])
# cntTf_app = pd.DataFrame(cntTf_app,
#                          columns=vectorizer.get_feature_names(),
#                          index=deviceid_packages.device)

'  (0, 19174)\t1\n  (0, 19760)\t1\n  (0, 3317)\t1\n  (1, 12416)\t1\n  (1, 14995)\t1\n  (1, 481)\t1\n  (1, 21901)\t1\n  (1, 25568)\t1\n  (1, 7243)\t1\n  (1, 21752)\t1\n  (1, 7597)\t1\n  (1, 6170)\t1\n  (1, 30950)\t1\n  (1, 31306)\t1\n  (1, 1459)\t1\n  (1, 19246)\t1\n  (1, 33185)\t1\n  (1, 25912)\t1\n  (1, 12838)\t1\n  (1, 20763)\t1\n  (1, 19174)\t1\n  (1, 3317)\t1\n  (2, 26794)\t1'

0        00009270c4ec26e1d76f5d86847009c9
1        000189ef5d5b951841d416a8c6c5b995
2        00026d79a6f0955fc860947724e24765
3        0002e3afb8146bc08e40575e45f0eca6
4        0004709a296f9b925ae283efe2f043e7
5        000536b155abcf97971a6b99a6d7dbef
6        0007c510ab41f8aedb2a759c2fce7aae
7        0007f90fc1f2da95e0b8c307e1598d21
8        000b216af4bf2cb7d8fe3cd66ddd03a8
9        000c6986f9c8431c12342193a978a582
10       000d87246382e5ed2888357424872081
11       000ef93f440c2dec1da90f25227cef1c
12       000efa9b64b19f672bf0963b76f3add9
13       000f30a1b1adf884c3f5cf8cb57b6f5c
14       000f3724e549428a979f20a9926be846
15       00105d9dc897b8365cf8c2a806c9ab33
16       00116cc2d1848d962760efb447064079
17       0013db8950e57afea8917d42aeb04d95
18       00176483b491cf4b06e1c5f5835b66a1
19       001831aa77641c0372eae802b0e7f77d
20       00187be1b35be18bdd38e24493a55fc0
21       001a4843fecc535dc69a2c7ef753b004
22       001a812beb1b7e8466602e5701bd635e
23       001a98576195f8ca110af7ced

In [ ]:
mini = get_start_closed()
tmp[tmp.device.isin(mini.device)][['device','duration_max_min', 'tol_day_cnt_nunique', 'gap']].sort_values('gap', ascending=False)


#f993126b72f6297440c2dc8534b20a0b  49
#fbf985012f537449b455d6a554479f8b  1161




In [ ]:
from  tiny.lda import *
get_lda_from_usage()

In [ ]:
device= 'fa68862f9484c62aca5c3a4b8a2c70bc'
mini[mini.device==device].sort_values('start')

In [ ]:
tmp[tmp.device==device]

In [ ]:
df2

In [ ]:
#df2.set_index('device', inplace=True)
df.sort_index(ascending=False)


In [ ]:
split.head()

In [ ]:
print(pd.to_datetime('1488332491588', unit='ms'))  #fc6383d2bdb463fd01db6731594a6d36	07e967d75aab2f6a52c558695a572a7c
print(pd.to_datetime('1488332506604', unit='ms'))
print(pd.to_datetime('1488363700875', unit='ms'))
print(pd.to_datetime('1488363707487', unit='ms'))

In [ ]:
tmp = extend_time(tmp,2)
#tmp['start'].astype
#tmp['start'] = pd.to_datetime(tmp['start'])
#tmp['start'].dt.date

In [ ]:

# def extend_percent(df):
#     total = get_percent_duration(df, ['device'], 'total')

#     max_week = get_max_week(df)

#     merge = df.merge(max_week, on=['device', 'weekbegin'])

#     max = get_percent_duration(merge, ['device'], prefix='max')

#     return pd.concat( [total, max], axis=1 )


# total = get_percent_duration(df, ['device'], 'total')


# df = tmp
# max_week = get_max_week(df)
# max_week.head()
# merge = df.merge(max_week, on=['device', 'weekbegin'])

 
# def get_percent_duration(df, groupby=['device', 'weekday'], prefix=None):
#     prefix = groupby[-1] if prefix is None else prefix
#     columns = [key for key in df.columns if 'span_' in key]
#     gp_map = [(key, 'sum') for key in columns if 'span_' in key]
#     gp_map = dict(gp_map)
#     # print(gp_map)

#     gp_map['package'] = 'nunique'
#     gp_map['start_base'] = 'nunique'
#     df = df.groupby(groupby).agg(gp_map)
#     #     df = df.groupby(groupby).agg({'span_0':'sum','span_1':'sum','span_2':'sum',
#     #                                                  'span_3':'sum','package':'nunique' })
#     df['total'] = df[[key for key in columns if 'span_' in key]].sum(axis=1)

#     for col in columns:
#         df[f'{col}_p'] = round(df[col] / df['total'], 3)

#     df.rename({'package':'package_cnt', 'start_base':'start_base_cnt'}, axis=1, inplace=True)

#     df.columns = [f'{prefix}_{key}' for key in df.columns]
#     return df

per = extend_percent(tmp)
per.head()

# max_week = get_max_week(tmp)
# max_week

# merge= tmp.merge(max_week, on = ['device','weekbegin'])

# print(len(merge))

# get_percent_duration(merge, ['device'], prefix='max')


    

In [ ]:
per
#tmp[tmp.device=='fc6383d2bdb463fd01db6731594a6d36'].sort_values('start')

In [ ]:
print('yes')

In [ ]:
#tmp['start_base']  = tmp['start'].dt.date
# tmp = tmp[tmp.device.isin(['fff885dafce62310951c835247f30a38',
# 'fc65f1731b785a8ce82d20a30dee9259'])]

df = tmp.groupby(['device','weekbegin']).package.nunique().to_frame().reset_index()

# .to_frame().reset_index()
# df.sort_values(['device', 'package'])

df.sort_values(by=['device', 'package', 'weekbegin'], ascending=False).groupby('device').nth(0)


In [ ]:
xx = df



In [ ]:


import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

path = './input/'
data = pd.DataFrame()
sex_age = pd.read_excel('./input/性别年龄对照表.xlsx')
deviceid_packages = pd.read_csv(path + 'deviceid_packages.tsv', sep='\t', names=['device', 'apps'])
deviceid_test = pd.read_csv(path + 'deviceid_test.tsv', sep='\t', names=['device'])
deviceid_train = pd.read_csv(path + 'deviceid_train.tsv', sep='\t', names=['device', 'sex', 'age'])

deviceid_train = pd.concat([deviceid_train, deviceid_test])

deviceid_packages['apps'] = deviceid_packages['apps'].apply(lambda x: x.split(','))
deviceid_packages['app_lenghth'] = deviceid_packages['apps'].apply(lambda x: len(x))

deviceid_packages.head()
# apps = deviceid_packages['apps'].apply(lambda x: ' '.join(x)).tolist()

# vectorizer = CountVectorizer()
# transformer = TfidfTransformer()
# cntTf = vectorizer.fit_transform(apps)
# tfidf = transformer.fit_transform(cntTf)
# #word = vectorizer.get_feature_names()
# weight = tfidf.toarray()
# df_weight = pd.DataFrame(weight)
# feature = df_weight.columns
# df_weight['sum'] = 0
# for f in tqdm(feature):
#     df_weight['sum'] += df_weight[f]
# deviceid_packages['tfidf_sum'] = df_weight['sum']

# lda = LatentDirichletAllocation(n_topics=5,
#                                 learning_offset=50.,
#                                 random_state=666)
# docres = lda.fit_transform(cntTf)

# deviceid_packages = pd.concat([deviceid_packages, pd.DataFrame(docres)], axis=1)

# temp = deviceid_packages.drop('apps', axis=1)
# deviceid_train = pd.merge(deviceid_train, temp, on='device', how='left')

# deviceid_train['sex'] = deviceid_train['sex'].apply(lambda x: str(x))
# deviceid_train['age'] = deviceid_train['age'].apply(lambda x: str(x))




In [ ]:
from tiny.lda import *
get_lda_from_usage()

In [ ]:
tmp = extend_cols(tmp)
tmp, tmp_new = split_days(tmp)

tmp.head()

In [ ]:
tmp, tmp_new = split_days(tmp)
tmp.head()

In [ ]:
trainn_label = get_train_label()
trainn_label.head()

In [ ]:
from  tiny.util import *

tmp = extend_feature( span_no=2, trunc_long_time=False, mini=True)

In [ ]:
tmp.head()

In [ ]:
tmp = extend_time_span(version=1, trunc_long_time=False, mini=True, groupby=['device'], prefix='tol')
tmp.head()

In [ ]:
from tiny.lda import *
from  tiny.util import *

tmp = extend_package(version=1)
tmp.head(3).sum(axis=1)

In [ ]:
tmp = tmp/tmp

tmp.head(3).sum(axis=1)

In [ ]:
tmp.head(3).count(axis=1)

In [ ]:
tmp.head(3)